<a href="https://colab.research.google.com/github/QingyunSun/tensorflow-wavenet/blob/master/Tseries_simple_process_on_stocknet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



> Process Data



In [0]:
#@title
universe = ['AAPL',
 'ABB',
 'ABBV',
 'AEP',
 'AGFS',
 'AMGN',
 'AMZN',
 'BA',
 'BABA',
 'BAC',
 'BBL',
 'BCH',
 'BHP',
 'BP',
 'BRK-A',
 'BSAC',
 'BUD',
 'C',
 'CAT',
 'CELG',
 'CHL',
 'CHTR',
 'CMCSA',
 'CODI',
 'CSCO',
 'CVX',
 'D',
 'DHR',
 'DIS',
 'DUK',
 'EXC',
 'FB',
 'GD',
 'GE',
 'GMRE',
 'GOOG',
 'HD',
 'HON',
 'HRG',
 'HSBC',
 'IEP',
 'INTC',
 'JNJ',
 'JPM',
 'KO',
 'LMT',
 'MA',
 'MCD',
 'MDT',
 'MMM',
 'MO',
 'MRK',
 'MSFT',
 'NEE',
 'NGG',
 'NVS',
 'ORCL',
 'PCG',
 'PCLN',
 'PEP',
 'PFE',
 'PG',
 'PICO',
 'PM',
 'PPL',
 'PTR',
 'RDS-B',
 'REX',
 'SLB',
 'SNP',
 'SNY',
 'SO',
 'SPLP',
 'SRE',
 'T',
 'TM',
 'TOT',
 'TSM',
 'UL',
 'UN',
 'UNH',
 'UPS',
 'UTX',
 'V',
 'VZ',
 'WFC',
 'WMT',
 'XOM']

In [0]:
from __future__ import print_function
import pandas as pd
import numpy as np
import datetime as dt
import xarray as xr
import sys, os

In [3]:
from google.colab import files
uploaded = files.upload()

Saving AAPL.csv to AAPL.csv
Saving ABB.csv to ABB.csv
Saving ABBV.csv to ABBV.csv
Saving AEP.csv to AEP.csv
Saving AGFS.csv to AGFS.csv
Saving AMGN.csv to AMGN.csv
Saving AMZN.csv to AMZN.csv
Saving BA.csv to BA.csv
Saving BABA.csv to BABA.csv
Saving BAC.csv to BAC.csv
Saving BBL.csv to BBL.csv
Saving BCH.csv to BCH.csv
Saving BHP.csv to BHP.csv
Saving BP.csv to BP.csv
Saving BRK-A.csv to BRK-A.csv
Saving BSAC.csv to BSAC.csv
Saving BUD.csv to BUD.csv
Saving C.csv to C.csv
Saving CAT.csv to CAT.csv
Saving CELG.csv to CELG.csv
Saving CHL.csv to CHL.csv
Saving CHTR.csv to CHTR.csv
Saving CMCSA.csv to CMCSA.csv
Saving CODI.csv to CODI.csv
Saving CSCO.csv to CSCO.csv
Saving CVX.csv to CVX.csv
Saving D.csv to D.csv
Saving DHR.csv to DHR.csv
Saving DIS.csv to DIS.csv
Saving DUK.csv to DUK.csv
Saving EXC.csv to EXC.csv
Saving FB.csv to FB.csv
Saving GD.csv to GD.csv
Saving GE.csv to GE.csv
Saving GMRE.csv to GMRE.csv
Saving GOOG.csv to GOOG.csv
Saving HD.csv to HD.csv
Saving HON.csv to HON.cs

In [0]:
import io
from collections import OrderedDict

data = OrderedDict()
for u in universe:
  file = '{u}.csv'.format(u=u)
  data[u] = pd.read_csv(io.BytesIO(uploaded[file]))

In [0]:
def extract_feature(df):
    log_adj_close_delta = np.log(df.iloc[1:]["Adj Close"] / df.iloc[:-1]["Adj Close"].values)
    log_volumn_delta = np.log(df.iloc[1:]["Volume"] / df.iloc[:-1]["Volume"].values)
    high_low = (df.iloc[1:]["High"] - df.iloc[1:]["Low"]) / (df.iloc[1:]["High"] + df.iloc[1:]["Low"]) * 2.0
    open_close = (df.iloc[1:]["Open"] - df.iloc[1:]["Close"]) / (df.iloc[1:]["Open"] + df.iloc[1:]["Close"]) * 2.0
    open_close_high_low = (df.iloc[1:]["Open"] - df.iloc[1:]["Close"]) / (df.iloc[1:]["High"] - df.iloc[1:]["Low"] + 1e-4)
    feature = pd.DataFrame(np.stack([log_adj_close_delta,
                        log_volumn_delta, high_low,
                        open_close, open_close_high_low], axis=-1), index=df.iloc[1:]["Date"])

    return feature

In [6]:
N_STOCK = len(universe)
total_days = 1258
days = data["AAPL"]["Date"][1:].values
N_FEATURE = 5

feature_data = xr.DataArray(np.zeros((total_days-1, N_STOCK, N_FEATURE)), 
                            dims = ("time", "symbols", "features"),
                            coords = {"time": days, "symbols": universe,
                                      "features": ["log_adj_close_delta", "log_volumn_delta", "high_low", "open_close", "open_close_high_low"]})
for st in universe:
  if data[st].shape[0] != total_days:
    print('{s} does not have enough days'.format(s = st))
  feature = extract_feature(data[st])
  feature_data.loc[feature.index, st, :] = feature.values

ABBV does not have enough days
AGFS does not have enough days
BABA does not have enough days


/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:679: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


GMRE does not have enough days


In [7]:
feature_data = feature_data.fillna(0)
feature_data[np.where(np.isinf(feature_data))] = 0.0
feature_data[np.where(feature_data > 6)] = 6.0
feature_data[np.where(feature_data < -6)] = -6.0
feature_data.shape

(1257, 88, 5)

In [8]:
FR=0
returns = feature_data[:, :, 0]
for ti in range(100):
    daily_FR = np.corrcoef(returns.values[ti, :], returns.values[ti+1, :])[0,1] * np.std(returns.values[ti+1, :])
    print(ti, daily_FR, np.corrcoef(returns.values[ti, :], returns.values[ti+1, :])[0,1])
    FR = FR + daily_FR
print(FR/100*1e4)

0 0.0006848670660560301 0.0596607160190975
1 0.006227054838831885 0.43215446104663197
2 0.00045971194211550184 0.04090395345704579
3 -0.0019156734239017962 -0.1706107672936251
4 0.0038648001895091565 0.33721493206320224
5 -0.0015196456744907146 -0.1542682515331423
6 0.0019170049617581507 0.12881035988492393
7 -0.004721294570210256 -0.4938241353376767
8 0.0012038698417875405 0.11671400601582987
9 0.0009425428377918865 0.10149431918992582
10 -0.0028201507268577307 -0.3177274956592921
11 4.413614627436219e-05 0.0048386348515283145
12 -0.0006107842400415625 -0.04667250288417307
13 0.003016912009301817 0.26205272213565134
14 0.0016745836558960225 0.1895709049337784
15 -0.005871644453214717 -0.5484816345625287
16 -0.002441054858359248 -0.19185883665569192
17 -0.00078373387768372 -0.09579265428936576
18 -0.0009037434950813448 -0.12847077157509232
19 -0.0009666244378869931 -0.10039072179171633
20 0.0016418584919365112 0.19006562062409865
21 -0.00170760024836933 -0.1896194831167164
22 -0.000139

In [0]:
WINDOW_SIZE = 20

def rolling_window_dataset(ti_start, ti_stop, feature_data):
  x = np.zeros((ti_stop-ti_start-WINDOW_SIZE, WINDOW_SIZE, N_STOCK, N_FEATURE))
  y = np.zeros((ti_stop-ti_start-WINDOW_SIZE, N_STOCK))
  for i in range(ti_start, ti_stop-WINDOW_SIZE):
    x[i-ti_start, :, :, :] = feature_data[i:i+WINDOW_SIZE, :, :].values
    y[i-ti_start, :] = feature_data[i+WINDOW_SIZE, :, :].loc[:, "log_adj_close_delta"].values

  return x, y

x_train, y_train = rolling_window_dataset(0, 760, feature_data)
x_val, y_val = rolling_window_dataset(760, 1010, feature_data)
x_test, y_test = rolling_window_dataset(1010, feature_data.shape[0]-1, feature_data)

> Model

In [0]:
import tensorflow.compat.v2 as tf
import tensorflow.compat.v1 as tfv1
tf.enable_v2_behavior()
tfv1.enable_v2_behavior()
from tensorflow.compat.v2 import keras
from tensorflow.compat.v2.keras import layers, optimizers, datasets
import tensorflow.compat.v2.keras.backend as K
import tensorflow_probability as tfp

import numpy as np
import os

Relative Position Embedding

In [0]:
#@title
@tf.function
def _generate_relative_positions_matrix(length, max_relative_position):
  """Generates matrix of relative positions between inputs."""
  range_vec = tf.range(length)
  range_mat = tf.reshape(tf.tile(range_vec, [length]), [length, length])
  distance_mat = range_mat - tf.transpose(range_mat)
  distance_mat_clipped = tf.clip_by_value(distance_mat, -max_relative_position,
                                          max_relative_position)
  # Shift values to be >= 0. Each integer still uniquely identifies a relative
  # position difference.
  final_mat = distance_mat_clipped + max_relative_position
  return final_mat

@tf.function
def _generate_relative_positions_embeddings(length,
                                            max_relative_position,
                                            embeddings_table):
  """Generates tensor of size [1 if cache else length, length, depth]."""
  relative_positions_matrix = _generate_relative_positions_matrix(
      length, max_relative_position)
  #vocab_size = max_relative_position * 2 + 1
  # Generates embedding for each relative position of dimension depth.
  #embeddings_table = tf.get_variable("embeddings", [vocab_size, depth])
  embeddings = tf.gather(embeddings_table, relative_positions_matrix)
  return embeddings

@tf.function
def _relative_attention_inner(x, y, z, transpose):
  """Relative position-aware dot-product attention inner calculation.
  This batches matrix multiply calculations to avoid unnecessary broadcasting.
  Args:
    x: Tensor with shape [batch_size, heads, length, length or depth].
    y: Tensor with shape [batch_size, heads, length, depth].
    z: Tensor with shape [length, length, depth].
    transpose: Whether to transpose inner matrices of y and z. Should be true if
        last dimension of x is depth, not length.
  Returns:
    A Tensor with shape [batch_size, heads, length, length or depth].
  """
  input_shape = K.shape(x)
  batch_size, heads, length = input_shape[0], input_shape[1], input_shape[2]

  # xy_matmul is [batch_size, heads, length, length or depth]
  xy_matmul = tf.matmul(x, y, transpose_b=transpose)
  # x_t is [length, batch_size, heads, length or depth]
  x_t = tf.transpose(x, [2, 0, 1, 3])
  # x_t_r is [length, batch_size * heads, length or depth]
  x_t_r = tf.reshape(x_t, [length, heads * batch_size, -1])
  # x_tz_matmul is [length batch_size * heads, length or depth]
  x_tz_matmul = tf.matmul(x_t_r, z, transpose_b=transpose)
  # x_tz_matmul_r is [length or 1, batch_size, heads, length or depth]
  x_tz_matmul_r = tf.reshape(x_tz_matmul, [length, batch_size, heads, -1])
  # x_tz_matmul_r_t is [batch_size, heads, length, length or depth]
  x_tz_matmul_r_t = tf.transpose(x_tz_matmul_r, [1, 2, 0, 3])
  return xy_matmul + x_tz_matmul_r_t

#_generate_relative_positions_matrix(WINDOW_SIZE, 5)
#etable = tf.Variable(initial_value=[np.arange(2*N_FEATURE+1)*0.1]*5, name="foo")
#_generate_relative_positions_embeddings(WINDOW_SIZE, 5, tf.transpose(etable))[:, :, 1]

Transformer


In [0]:
#@title
import math

@tf.function
def gelu(x):
  """An approximation of gelu.
  See: https://arxiv.org/pdf/1606.08415.pdf
  """
  return 0.5 * x * (1.0 + K.tanh(math.sqrt(2.0 / math.pi) * (x + 0.044715 * K.pow(x, 3))))

class LayerNormalization(keras.layers.Layer):
  def __init__(self,
                center=True,
                scale=True,
                epsilon=None,
                gamma_initializer='ones',
                beta_initializer='zeros',
                gamma_regularizer=None,
                beta_regularizer=None,
                gamma_constraint=None,
                beta_constraint=None,
                **kwargs):
      """Layer normalization layer
      See: [Layer Normalization](https://arxiv.org/pdf/1607.06450.pdf)
      :param center: Add an offset parameter if it is True.
      :param scale: Add a scale parameter if it is True.
      :param epsilon: Epsilon for calculating variance.
      :param gamma_initializer: Initializer for the gamma weight.
      :param beta_initializer: Initializer for the beta weight.
      :param gamma_regularizer: Optional regularizer for the gamma weight.
      :param beta_regularizer: Optional regularizer for the beta weight.
      :param gamma_constraint: Optional constraint for the gamma weight.
      :param beta_constraint: Optional constraint for the beta weight.
      :param kwargs:
      """
      super(LayerNormalization, self).__init__(**kwargs)
      self.supports_masking = False
      self.center = center
      self.scale = scale
      if epsilon is None:
          epsilon = K.epsilon() * K.epsilon()
      self.epsilon = epsilon
      self.gamma_initializer = keras.initializers.get(gamma_initializer)
      self.beta_initializer = keras.initializers.get(beta_initializer)
      self.gamma_regularizer = keras.regularizers.get(gamma_regularizer)
      self.beta_regularizer = keras.regularizers.get(beta_regularizer)
      self.gamma_constraint = keras.constraints.get(gamma_constraint)
      self.beta_constraint = keras.constraints.get(beta_constraint)
      self.gamma, self.beta = None, None

  def get_config(self):
      config = {
          'center': self.center,
          'scale': self.scale,
          'epsilon': self.epsilon,
          'gamma_initializer': keras.initializers.serialize(self.gamma_initializer),
          'beta_initializer': keras.initializers.serialize(self.beta_initializer),
          'gamma_regularizer': keras.regularizers.serialize(self.gamma_regularizer),
          'beta_regularizer': keras.regularizers.serialize(self.beta_regularizer),
          'gamma_constraint': keras.constraints.serialize(self.gamma_constraint),
          'beta_constraint': keras.constraints.serialize(self.beta_constraint),
      }
      base_config = super(LayerNormalization, self).get_config()
      return dict(list(base_config.items()) + list(config.items()))

  def compute_output_shape(self, input_shape):
      return input_shape

  def compute_mask(self, inputs, input_mask=None):
      return input_mask

  def build(self, input_shape):
      self.input_spec = keras.layers.InputSpec(shape=input_shape)
      shape = input_shape[-1:]
      if self.scale:
          self.gamma = self.add_weight(
              shape=shape,
              initializer=self.gamma_initializer,
              regularizer=self.gamma_regularizer,
              constraint=self.gamma_constraint,
              name='gamma',
          )
      if self.center:
          self.beta = self.add_weight(
              shape=shape,
              initializer=self.beta_initializer,
              regularizer=self.beta_regularizer,
              constraint=self.beta_constraint,
              name='beta',
          )
      super(LayerNormalization, self).build(input_shape)

  def call(self, inputs, training=None):
    mean = K.mean(inputs, axis=-1, keepdims=True)
    variance = K.mean(K.square(inputs - mean), axis=-1, keepdims=True)
    std = K.sqrt(variance + self.epsilon)
    outputs = (inputs - mean) / std
    if self.scale:
        outputs *= self.gamma
    if self.center:
        outputs += self.beta

    return outputs

class MultiHeadAttentionNoWeight(keras.layers.Layer):
  def __init__(self, **kwargs):
    self.activation = gelu
    self.supports_masking = False
    super(MultiHeadAttentionNoWeight, self).__init__(**kwargs)

  def build(self, input_shape):
    super(MultiHeadAttentionNoWeight, self).build(input_shape)

  def call(self, inputs):
    input_shape = K.shape(inputs)
    #batch_size, seq_len, head_num, head_dim = inputs[0], inputs[1], inputs[2], inputs[3]
    #query = K.reshape(inputs, (batch_size * seq_len, head_num, head_dim))
    #key = K.reshape(inputs, (batch_size * seq_len, head_num, head_dim))
    #value = K.reshape(inputs, (batch_size * seq_len, head_num, head_dim))
    query = key = value = inputs
    e = K.batch_dot(query, key, axes=3) / K.sqrt(K.cast(N_FEATURE, dtype=K.floatx()))
    e = K.exp(e - K.max(e, axis=-1, keepdims=True))
    a = e / (K.sum(e, axis=-1, keepdims=True) + K.epsilon())
    y = K.batch_dot(a, value)
    
    y = self.activation(y)

    return y
  
class MultiHeadAttention(keras.layers.Layer):
    """Multi-head attention layer.
    See: https://arxiv.org/pdf/1706.03762.pdf
    """

    def __init__(self,
                 head_num,
                 max_relative_position=5,
                 use_bias=True,
                 kernel_initializer='glorot_normal',
                 bias_initializer='zeros',
                 kernel_regularizer=None,
                 bias_regularizer=None,
                 kernel_constraint=None,
                 bias_constraint=None,
                 history_only=False,
                 **kwargs):
        """Initialize the layer.
        :param head_num: Number of heads.
        :param use_bias: Whether to use bias term.
        :param kernel_initializer: Initializer for linear mappings.
        :param bias_initializer: Initializer for linear mappings.
        :param kernel_regularizer: Regularizer for linear mappings.
        :param bias_regularizer: Regularizer for linear mappings.
        :param kernel_constraint: Constraints for linear mappings.
        :param bias_constraint: Constraints for linear mappings.
        :param history_only: Whether to only use history in attention layer.
        """
        self.supports_masking = False
        self.head_num = head_num
        self.max_relative_position = max_relative_position
        self.activation = gelu
        self.use_bias = use_bias
        self.kernel_initializer = keras.initializers.get(kernel_initializer)
        self.bias_initializer = keras.initializers.get(bias_initializer)
        self.kernel_regularizer = keras.regularizers.get(kernel_regularizer)
        self.bias_regularizer = keras.regularizers.get(bias_regularizer)
        self.kernel_constraint = keras.constraints.get(kernel_constraint)
        self.bias_constraint = keras.constraints.get(bias_constraint)
        self.history_only = history_only

        self.Wq, self.Wk, self.Wv, self.Wo = None, None, None, None
        self.bq, self.bk, self.bv, self.bo = None, None, None, None
        super(MultiHeadAttention, self).__init__(**kwargs)

    def get_config(self):
        config = {
            'head_num': self.head_num,
            'max_relative_position': self.max_relative_position,
            'use_bias': self.use_bias,
            'kernel_initializer': keras.initializers.serialize(self.kernel_initializer),
            'bias_initializer': keras.initializers.serialize(self.bias_initializer),
            'kernel_regularizer': keras.regularizers.serialize(self.kernel_regularizer),
            'bias_regularizer': keras.regularizers.serialize(self.bias_regularizer),
            'kernel_constraint': keras.constraints.serialize(self.kernel_constraint),
            'bias_constraint': keras.constraints.serialize(self.bias_constraint),
            'history_only': self.history_only,
        }
        base_config = super(MultiHeadAttention, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def compute_output_shape(self, input_shape):
        if isinstance(input_shape, list):
            q, k, v = input_shape
            return q[:-1] + (v[-1],)
        return input_shape

    def compute_mask(self, inputs, input_mask=None):
        if isinstance(input_mask, list):
            return input_mask[0]
        return input_mask

    def build(self, input_shape):
        new_feature_dim = WINDOW_SIZE * N_FEATURE
        self.Wq = self.add_weight(
            shape=(new_feature_dim, new_feature_dim),
            initializer=self.kernel_initializer,
            regularizer=self.kernel_regularizer,
            constraint=self.kernel_constraint,
            name='%s_Wq' % self.name,
        )
        if self.use_bias:
            self.bq = self.add_weight(
                shape=(new_feature_dim,),
                initializer=self.bias_initializer,
                regularizer=self.bias_regularizer,
                constraint=self.bias_constraint,
                name='%s_bq' % self.name,
            )
        self.Wk = self.add_weight(
            shape=(new_feature_dim, new_feature_dim),
            initializer=self.kernel_initializer,
            regularizer=self.kernel_regularizer,
            constraint=self.kernel_constraint,
            name='%s_Wk' % self.name,
        )
        if self.use_bias:
            self.bk = self.add_weight(
                shape=(new_feature_dim,),
                initializer=self.bias_initializer,
                regularizer=self.bias_regularizer,
                constraint=self.bias_constraint,
                name='%s_bk' % self.name,
            )
        self.Wv = self.add_weight(
            shape=(new_feature_dim, new_feature_dim),
            initializer=self.kernel_initializer,
            regularizer=self.kernel_regularizer,
            constraint=self.kernel_constraint,
            name='%s_Wv' % self.name,
        )
        if self.use_bias:
            self.bv = self.add_weight(
                shape=(new_feature_dim,),
                initializer=self.bias_initializer,
                regularizer=self.bias_regularizer,
                constraint=self.bias_constraint,
                name='%s_bv' % self.name,
            )
        self.Wo = self.add_weight(
            shape=(new_feature_dim, new_feature_dim),
            initializer=self.kernel_initializer,
            regularizer=self.kernel_regularizer,
            constraint=self.kernel_constraint,
            name='%s_Wo' % self.name,
        )
        if self.use_bias:
            self.bo = self.add_weight(
                shape=(new_feature_dim,),
                initializer=self.bias_initializer,
                regularizer=self.bias_regularizer,
                constraint=self.bias_constraint,
                name='%s_bo' % self.name,
            )
        self.position_emb_table = self.add_weight(
            shape=(2*self.max_relative_position+1, N_FEATURE),
            initializer=self.kernel_initializer,
            regularizer=self.kernel_regularizer,
            constraint=self.kernel_constraint,
            name='%s_PositionEmbTable' % self.name,            
        )
        super(MultiHeadAttention, self).build(input_shape)

    @staticmethod
    def _reshape_to_batches(x, head_num):
        input_shape = K.shape(x)
        batch_size, seq_len, feature_dim = input_shape[0], input_shape[1], input_shape[2]
        head_dim = feature_dim // head_num
        x = K.reshape(x, (batch_size, seq_len, head_num, head_dim))
        x = K.permute_dimensions(x, [0, 2, 1, 3])
        return x #K.reshape(x, (batch_size * head_num, seq_len * head_dim))

    @staticmethod
    def _reshape_from_batches(x, head_num):
        input_shape = K.shape(x)
        batch_size, seq_len, feature_dim = input_shape[0], input_shape[1], input_shape[2]
        x = K.reshape(x, (batch_size // head_num, head_num, seq_len, feature_dim))
        x = K.permute_dimensions(x, [0, 2, 1, 3])
        return K.reshape(x, (batch_size // head_num, seq_len, feature_dim * head_num))

    @staticmethod
    def _reshape_mask(mask, head_num):
        if mask is None:
            return mask
        seq_len = K.shape(mask)[1]
        mask = K.expand_dims(mask, axis=1)
        mask = K.tile(mask, [1, head_num, 1])
        return K.reshape(mask, (-1, seq_len))

    def call(self, inputs):
        batch_size = K.shape(inputs)[0]
        x = K.permute_dimensions(inputs, [0, 2, 1, 3])
        x = K.reshape(x, (batch_size, N_STOCK, WINDOW_SIZE * N_FEATURE))
        q = k = v = x
        q = K.dot(q, self.Wq)
        k = K.dot(k, self.Wk)
        v = K.dot(v, self.Wv)
        if self.use_bias:
            q += self.bq
            k += self.bk
            v += self.bv
        if self.activation is not None:
            q = self.activation(q)
            k = self.activation(k)
            v = self.activation(v)

        position_emb = _generate_relative_positions_embeddings(
            WINDOW_SIZE, self.max_relative_position, self.position_emb_table)
        query = K.reshape(q, (batch_size, N_STOCK, WINDOW_SIZE, N_FEATURE))
        key = K.reshape(k, (batch_size, N_STOCK, WINDOW_SIZE, N_FEATURE))
        value = K.reshape(v, (batch_size, N_STOCK, WINDOW_SIZE, N_FEATURE))
        
        e = _relative_attention_inner(query, key, position_emb, True)
        e = K.exp(e - K.max(e, axis=-1, keepdims=True)) # [B, H, L, L]
        a = e / (K.sum(e, axis=-1, keepdims=True) + K.epsilon())
        y = K.batch_dot(a, value)
        y = K.reshape(y, (batch_size, N_STOCK, WINDOW_SIZE * N_FEATURE))
        
        y = K.dot(y, self.Wo)
        if self.use_bias:
            y += self.bo
        if self.activation is not None:
            y = self.activation(y)
            
        y = K.reshape(y, (batch_size, N_STOCK, WINDOW_SIZE, N_FEATURE))
        y = K.permute_dimensions(y, [0, 2, 1, 3])
        return y

Full model

In [35]:
class TsTransformer(keras.Model):
    def __init__(self):
        super(TsTransformer, self).__init__()

#        self.attn_xs = MultiHeadAttentionNoWeight()
        # self.attn_ts = MultiHeadAttention(head_num=N_STOCK, max_relative_position=5)
        # self.dropout = keras.layers.Dropout(rate=0.3)
        # self.layer_norm = LayerNormalization()
        self.fc = keras.layers.Dense(N_STOCK, input_shape=(N_STOCK*N_FEATURE,))

    def call(self, inputs, training=None, mask=None):
      # inputs is expected to have the shape (batch_size, WINDOW_SIZE, N_STOCK, N_FEATURE)
      x = inputs
      input_shape = K.shape(x)
      batch_size = input_shape[0]
      for i in range(2):
        input_x = x
 #       x = self.attn_xs(x)
        # x = self.attn_ts(x)
        # x = self.fc(x)
        # x = self.dropout(x)
        # x = input_x + x
        # x = self.layer_norm(x)
        # input_x = x
        # x = self.attn_ts(x)
        # x = self.fc(x)
        # x = self.dropout(x)
        # x = input_x + x
        # x = self.layer_norm(x)
      x = self.fc(K.reshape(x[:, -1, :], (batch_size, N_STOCK * N_FEATURE)))

      return x

model = TsTransformer()
model.build(input_shape=(None, WINDOW_SIZE, N_STOCK, N_FEATURE))
model.summary()

Model: "ts_transformer_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              multiple                  38808     
Total params: 38,808
Trainable params: 38,808
Non-trainable params: 0
_________________________________________________________________


In [0]:
tfv1.reset_default_graph()

In [0]:
#logs_base_dir = "./logs"
#os.makedirs(logs_base_dir, exist_ok=True)
#%load_ext tensorboard
#%tensorboard --logdir {logs_base_dir}

In [37]:
batch_size = 32
epochs = 3

@tf.function
def compute_loss(y_pred, y):
  y = tf.cast(y, tf.float32)
  return -tf.reduce_mean(tfp.stats.correlation(y_pred, y, sample_axis=1, event_axis=None) * tfp.stats.stddev(y, sample_axis=1))

optimizer = keras.optimizers.Adam(0.001)

if False:
  model.compile(optimizer=optimizer,
                loss=keras.losses.MeanSquaredError())

  logdir = os.path.join(logs_base_dir, dt.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  # train
  model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs,
            validation_data=(x_test, y_test), callbacks=[tensorboard_callback],
            verbose=1)

  # evaluate on test set
  scores = model.evaluate(x_test, y_test, batch_size, verbose=1)
  print("Final test loss and accuracy :", scores)
else:
  count = 0; avg_loss = None; loss_buffer = []; stop_flag = False
  for epoch in range(epochs):
    ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    ds = ds.shuffle(x_train.shape[0]).batch(batch_size)
    for step, (xs, ys) in enumerate(ds):
      with tf.GradientTape() as tape:
        y_pred = model(tf.cast(xs, tf.float32))
        loss = compute_loss(y_pred, ys)

      # compute gradient
      grads = tape.gradient(loss, model.trainable_variables)
      grads = [tf.where(tf.math.is_nan(g), tf.zeros_like(g), g) for g in grads]
      # update to weights
      optimizer.apply_gradients(zip(grads, model.trainable_variables))
      val_loss = float(compute_loss(model(tf.cast(x_val, tf.float32)), y_val))*1e4
      print(epoch, step, float(loss)*1e4, val_loss)
      count = count + 1; loss_buffer.append(val_loss)
      if count % 5 == 0:
        new_avg_loss = np.mean(np.array(loss_buffer))
        #if avg_loss is not None and new_avg_loss > avg_loss:
        #  stop_flag = True
        #  break
        loss_buffer = []
        avg_loss = new_avg_loss
    if stop_flag:
      break

0 0 2.330923598492518 2.174522669520229
0 1 -0.5660254100803286 2.134860696969554
0 2 -1.9029091345146298 2.090511698042974
0 3 -1.6445523942820728 2.0375847816467285
0 4 -2.025749417953193 1.9710765627678484
0 5 -3.653267049230635 1.9165920093655586
0 6 -2.3035830236040056 1.8662717775441706
0 7 2.835286722984165 1.8392714264336973
0 8 3.198060439899564 1.823230559239164
0 9 1.6192867769859731 1.8092765822075307
0 10 -0.9400833368999884 1.8039523274637759
0 11 -2.1051321527920663 1.7962830315809697
0 12 1.490817521698773 1.7822244262788445
0 13 2.2022398479748517 1.762453030096367
0 14 0.00398969859816134 1.749138900777325
0 15 -1.0807072976604104 1.731012889649719
0 16 0.07396716682706028 1.7088261665776372
0 17 -1.1802566586993635 1.6826855426188558
0 18 1.3026021770201623 1.6369210788980126
0 19 0.039498991100117564 1.5939236618578434
0 20 -2.455766953062266 1.5597291348967701
0 21 -1.6023848729673773 1.5349630848504603
0 22 3.3307509147562087 1.5225289098452777
0 23 -0.57014229241

(740, 20, 88, 5)

In [31]:
-compute_loss(model(x_test), tf.cast(y_test, tf.float32))*1e4


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



<tf.Tensor: shape=(), dtype=float32, numpy=0.737475>

In [38]:
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

y_hat = model(x_test)
X = np.expand_dims(np.array(y_hat).flatten(), axis=-1)
y = np.array(y_test).flatten()
reg = LinearRegression(fit_intercept=True).fit(X, y)
np.sqrt(reg.score(X, y))


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



0.004270088424607299

In [39]:
np.sqrt(reg.score(X, y)) * np.std(y) * 1e4

0.5556694111285609